In [9]:
import requests

print(requests.__version__)  # This prints the version of the requests library
print(dir(requests.exceptions))  # This prints the directory of the requests.exceptions module, verifying its existence



2.31.0
['BaseHTTPError', 'ChunkedEncodingError', 'CompatJSONDecodeError', 'ConnectTimeout', 'ConnectionError', 'ContentDecodingError', 'FileModeWarning', 'HTTPError', 'InvalidHeader', 'InvalidJSONError', 'InvalidProxyURL', 'InvalidSchema', 'InvalidURL', 'JSONDecodeError', 'MissingSchema', 'ProxyError', 'ReadTimeout', 'RequestException', 'RequestsDependencyWarning', 'RequestsWarning', 'RetryError', 'SSLError', 'StreamConsumedError', 'Timeout', 'TooManyRedirects', 'URLRequired', 'UnrewindableBodyError', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__spec__']


In [10]:
import ee

# Initialize the Earth Engine library.
ee.Initialize()


In [11]:
# Fetch a Landsat 8 image.
image = ee.Image('LANDSAT/LC08/C01/T1/LC08_044034_20140318')

# Print the image metadata.
print(image.getInfo())


{'type': 'Image', 'bands': [{'id': 'B1', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 65535}, 'dimensions': [7661, 7801], 'crs': 'EPSG:32610', 'crs_transform': [30, 0, 460785, 0, -30, 4264215]}, {'id': 'B2', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 65535}, 'dimensions': [7661, 7801], 'crs': 'EPSG:32610', 'crs_transform': [30, 0, 460785, 0, -30, 4264215]}, {'id': 'B3', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 65535}, 'dimensions': [7661, 7801], 'crs': 'EPSG:32610', 'crs_transform': [30, 0, 460785, 0, -30, 4264215]}, {'id': 'B4', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 65535}, 'dimensions': [7661, 7801], 'crs': 'EPSG:32610', 'crs_transform': [30, 0, 460785, 0, -30, 4264215]}, {'id': 'B5', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 65535}, 'dimensions': [7661, 7801], 'crs': 'EPSG:32610', 'crs_transform': [30, 0, 460785, 0, -30, 4264215]}, {'

In [12]:
#import folium` is importing the Folium library in Python. Folium is a Python library used for visualizing geospatial data interactively on an interactive map.
import folium
import ee

ee.Initialize()

# Define a function to add Earth Engine layers to a Folium map.
def add_ee_layer(self, ee_image_object, vis_params, name):
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Map Data © Google Earth Engine',
        name=name,
        overlay=True,
        control=True
    ).add_to(self)

# Add Earth Engine layer method to folium.Map.
folium.Map.add_ee_layer = add_ee_layer

# Example usage
# Define a location: Johannesburg, South Africa
location = [-26.2041, 28.0473]

# Create a folium map object.
my_map = folium.Map(location=location, zoom_start=10)

# Fetch a Landsat 8 image.
image = ee.Image('LANDSAT/LC08/C01/T1/LC08_044034_20140318')

# Set visualization parameters.
vis_params = {
    'bands': ['B4', 'B3', 'B2'],  # RGB bands
    'min': 0,
    'max': 30000,
    'gamma': 1.4
}

# Add the Landsat 8 image to the map object.
my_map.add_ee_layer(image, vis_params, 'Landsat 8')

# Add a layer control panel to the map.
my_map.add_child(folium.LayerControl())

# Display the map.
my_map





In [13]:
import ee
ee.Initialize()

# Define the area of interest: Johannesburg, South Africa.
aoi = ee.Geometry.Point([28.0473, -26.2041]).buffer(10000)  # Buffer by 10 km

# Load datasets.
lst = ee.ImageCollection('MODIS/006/MOD11A1').select('LST_Day_1km').mean().clip(aoi)  # Mean LST
ndvi = ee.ImageCollection('MODIS/006/MOD13Q1').select('NDVI').mean().clip(aoi)  # Mean NDVI
population = ee.Image("WorldPop/GP/100m/pop").clip(aoi)  # Population density

# Normalize datasets (simplified example, actual normalization depends on data range).
lst_norm = lst.subtract(lst.reduceRegion(ee.Reducer.min(), aoi, 1000).get('LST_Day_1km')).divide(
    lst.reduceRegion(ee.Reducer.max(), aoi, 1000).get('LST_Day_1km').subtract(lst.reduceRegion(ee.Reducer.min(), aoi, 1000).get('LST_Day_1km'))
)
ndvi_norm = ndvi.subtract(ndvi.reduceRegion(ee.Reducer.min(), aoi, 1000).get('NDVI')).divide(
    ndvi.reduceRegion(ee.Reducer.max(), aoi, 1000).get('NDVI').subtract(ndvi.reduceRegion(ee.Reducer.min(), aoi, 1000).get('NDVI'))
)
pop_norm = population.divide(population.reduceRegion(ee.Reducer.max(), aoi, 1000).get('population'))

# Assuming equal weighting, create a Heat Vulnerability Index (HVI).
# Note: This is a simplified approach. Weights can be adjusted based on local studies or expert input.
hvi = lst_norm.add(ndvi_norm.multiply(-1)).add(pop_norm).divide(3)

# Visualization parameters.
vis_params = {
    'min': 0,
    'max': 1,
    'palette': ['green', 'yellow', 'red']
}

# Create a Folium map with the HVI layer.
import folium

def add_ee_layer(self, ee_image_object, vis_params, name):
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Map Data © Google Earth Engine',
        name=name,
        overlay=True,
        control=True
    ).add_to(self)

folium.Map.add_ee_layer = add_ee_layer

map = folium.Map(location=[-26.2041, 28.0473], zoom_start=10)
map.add_ee_layer(hvi, vis_params, 'Heat Vulnerability Index')
map.add_child(folium.LayerControl())

map


AttributeError: 'ComputedObject' object has no attribute 'subtract'

In [ ]:
# Assuming `lst`, `ndvi`, and `population` are defined as per your snippet.

# Convert the reductions to numbers (Python side - not recommended for production due to potential performance issues).
min_lst = ee.Number(lst.reduceRegion(ee.Reducer.min(), aoi, 1000).get('LST_Day_1km')).getInfo()
max_lst = ee.Number(lst.reduceRegion(ee.Reducer.max(), aoi, 1000).get('LST_Day_1km')).getInfo()

# Now, min_lst and max_lst are Python numbers, and you can use them in calculations directly.
# For Earth Engine operations, you'd want to keep those as ee.Number for any server-side calculations.


In [ ]:
# Correcting the approach to stay server-side
lst_norm = lst.subtract(ee.Number(lst.reduceRegion(ee.Reducer.min(), aoi, 1000).get('LST_Day_1km'))).divide(
    ee.Number(lst.reduceRegion(ee.Reducer.max(), aoi, 1000).get('LST_Day_1km')).subtract(ee.Number(lst.reduceRegion(ee.Reducer.min(), aoi, 1000).get('LST_Day_1km')))
)

# Similarly for NDVI and population, ensure operations are performed with ee.Number or directly on images.


In [ ]:
import ee
import folium

ee.Initialize()

# Define the area of interest: Johannesburg, South Africa.
aoi = ee.Geometry.Point([28.0473, -26.2041]).buffer(10000)  # Buffer by 10 km

# Load datasets and mean over the specified period for simplicity.
lst = ee.ImageCollection('MODIS/006/MOD11A1').select('LST_Day_1km').mean().clip(aoi)  # Mean LST
ndvi = ee.ImageCollection('MODIS/006/MOD13Q1').select('NDVI').mean().clip(aoi)  # Mean NDVI
# Example for South Africa, year 2020
#population = ee.Image("WorldPop/GP/100m/ZAF_2020").clip(aoi)

# Normalize each dataset. The normalization is performed by subtracting the minimum
# of the dataset and dividing by the range of the dataset.
# These operations are kept server-side.
def normalize(image, region, scale=1000):
    minVal = ee.Number(image.reduceRegion(ee.Reducer.min(), region, scale).values().get(0))
    maxVal = ee.Number(image.reduceRegion(ee.Reducer.max(), region, scale).values().get(0))
    return image.subtract(minVal).divide(maxVal.subtract(minVal))

lst_norm = normalize(lst, aoi)
ndvi_norm = normalize(ndvi, aoi)
#pop_norm = normalize(population, aoi)

# Combine the normalized factors to create the HVI. Here, NDVI is subtracted because higher NDVI values
# (more vegetation) typically mean lower heat vulnerability, contrary to LST and population density.
# This is a simple arithmetic mean of the factors to create the index.
hvi = lst_norm.add(ndvi_norm.multiply(-1)).divide(2)

# Visualization parameters for HVI.
vis_params = {
    'min': 0,
    'max': 1,
    'palette': ['blue', 'white', 'red']
}

# Function to add Earth Engine layers to Folium map.
def add_ee_layer(self, ee_image_object, vis_params, name):
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Map Data © Google Earth Engine',
        name=name,
        overlay=True,
        control=True
    ).add_to(self)

folium.Map.add_ee_layer = add_ee_layer

# Create a Folium map centered on the area of interest.
map = folium.Map(location=[-26.2041, 28.0473], zoom_start=10)
map.add_ee_layer(hvi, vis_params, 'Heat Vulnerability Index')

# Add a layer control panel to toggle layers on and off.
map.add_child(folium.LayerControl())

# Display the map.
map


In [ ]:
import folium
import ee

ee.Initialize()

# Define the area of interest: Johannesburg, South Africa.
aoi = ee.Geometry.Point([28.0473, -26.2041]).buffer(10000)  # Buffer by 10 km

# Function to add Earth Engine layers to Folium map.
def add_ee_layer(self, ee_image_object, vis_params, name):
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Map Data © Google Earth Engine',
        name=name,
        overlay=True,
        control=True
    ).add_to(self)

# Add Earth Engine layer method to folium.Map.
folium.Map.add_ee_layer = add_ee_layer

# Example known-good layer (MODIS NDVI)
modis_ndvi = ee.ImageCollection('MODIS/006/MOD13Q1').select('NDVI').mean().clip(aoi)
ndvi_vis_params = {
    'min': 0,
    'max': 9000,
    'palette': ['00FF00', 'FF0000']
}

# Create a Folium map centered on the area of interest.
simple_map = folium.Map(location=[-26.2041, 28.0473], zoom_start=10)

# Add the MODIS NDVI layer to the map using the corrected variable name for visualization parameters.
simple_map.add_ee_layer(modis_ndvi, ndvi_vis_params, 'MODIS NDVI')

# Display the simplified map.
simple_map.add_child(folium.LayerControl())
simple_map



In [ ]:
# Attempt to get Johannesburg boundaries from GADM data (administrative boundaries)
# Note: Adjust "ZAF" and administrative level as necessary to better fit Johannesburg boundaries.
johannesburg_boundaries = ee.FeatureCollection('FAO/GAUL/2015/level2').filter(ee.Filter.eq('ADM2_NAME', 'Johannesburg'))

# To visualize and confirm boundaries, you might want to display this layer first.


In [ ]:
# Assuming `hvi` is your final Heat Vulnerability Index image:
hvi_johannesburg = hvi.clip(johannesburg_boundaries)


In [ ]:
# Assuming `lst`, `ndvi`, and possibly `population` are defined earlier
try:
    # Check if lst_norm evaluates to an image
    print(lst_norm.getInfo())
    
    # Check if ndvi_norm evaluates to an image
    print(ndvi_norm.getInfo())
    
    # If using population data
    # print(pop_norm.getInfo())
except Exception as e:
    print(e)


{'type': 'Image', 'bands': [{'id': 'LST_Day_1km', 'data_type': {'type': 'PixelType', 'precision': 'double', 'min': -83.22128864612405, 'max': 283.3800330045181}, 'dimensions': [2, 2], 'origin': [27, -27], 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}], 'properties': {'system:footprint': {'type': 'Polygon', 'coordinates': [[[28.0473, -26.114111340485184], [28.019306616602588, -26.117690180576375], [27.99353669130437, -26.128142452689524], [27.972038190924838, -26.144637843859336], [27.956521606502278, -26.165865545252068], [27.948224048435236, -26.19013790088324], [27.94781015862496, -26.21552409985472], [27.955317913685608, -26.240003471206084], [27.97015403235698, -26.261626304440053], [27.991139842273643, -26.27866940413439], [28.01660438259738, -26.289773886250117], [28.04451756112857, -26.29405405080564], [28.072652723210197, -26.291168442906578], [28.098765387075858, -26.281347269201113], [28.120773447805355, -26.265373912599795], [28.136924019334938, -26.24452206754058

In [ ]:
# Assuming `johannesburg_boundaries` is a FeatureCollection outlining the city or wards
hvi_johannesburg = hvi.clip(johannesburg_boundaries)

# Visualization parameters, adjusted for NDVI as an example
ndvi_vis_params = {
    'min': 0.2,  # Adjust based on your NDVI data to enhance visibility
    'max': 0.8,
    'palette': ['blue', 'white', 'green']
}

# Initialize a folium map centered on Johannesburg
map_johannesburg = folium.Map(location=[-26.2041, 28.0473], zoom_start=11)

# Add the NDVI layer with enhanced visibility
map_johannesburg.add_ee_layer(modis_ndvi.clip(johannesburg_boundaries), ndvi_vis_params, 'Enhanced NDVI')

# Add the HVI layer
map_johannesburg.add_ee_layer(hvi_johannesburg, {
    'min': 0,
    'max': 1,
    'palette': ['yellow', 'red']
}, 'Heat Vulnerability Index')

# Optionally, add Johannesburg boundaries for reference
# Ensure 'johannesburg_boundaries' is styled appropriately for visibility
map_johannesburg.add_ee_layer(johannesburg_boundaries.style(**{
    'color': 'black',
    'fillColor': '00000000'  # Transparent fill
}), {}, 'Johannesburg Boundaries')

# Add a layer control panel
map_johannesburg.add_child(folium.LayerControl())

# Display the map
map_johannesburg



In [ ]:
import folium
import ee

ee.Initialize()

# Define your area of interest or load Johannesburg boundaries
# For demonstration, let's assume johannesburg_boundaries is already defined

# Create an empty image onto which the boundaries will be drawn.
empty = ee.Image().byte()

# Draw the boundaries as lines on the empty image.
boundary_image = empty.paint(**{
    'featureCollection': johannesburg_boundaries,
    'color': 1,  # This is a nominal value for the paint operation.
    'width': 3  # Width of the outline.
})

# Set visualization parameters for the boundaries.
vis_params = {
    'palette': ['red'],
    'min': 0,
    'max': 3,
}

# Initialize a Folium map centered on Johannesburg.
map_johannesburg = folium.Map(location=[-26.2041, 28.0473], zoom_start=10)

# Define the function to add Earth Engine layers to Folium map if not already defined.
def add_ee_layer(self, ee_image_object, vis_params, name):
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Map Data © Google Earth Engine',
        name=name,
        overlay=True,
        control=True
    ).add_to(self)

# Add this method to folium.Map objects
folium.Map.add_ee_layer = add_ee_layer

# Now, use this function to add the boundary image as a layer.
map_johannesburg.add_ee_layer(boundary_image, vis_params, 'Johannesburg Boundaries')

# Add a layer control panel.
map_johannesburg.add_child(folium.LayerControl())

# Display the map.
map_johannesburg

